In [1]:
import pandas as pd
import glob
import os
from openpyxl import load_workbook, Workbook
import openpyxl
import re
from datetime import datetime

In [2]:
# These are the O drive location where we'll save our files
path = r"\\fpcl01\public\OFWSHARE\PR24\231002 - Business Plan Submissions\1_Files as submitted by companies\Core"
path2 = r"O:\OFWSHARE\PR24\231113 - Business Plan Submissions\2_Files after Panacea\Core\Amended files"
path3 = r"O:\OFWSHARE\PR24\231113 - Business Plan Submissions\3_Files error fixing WORKING\Core"

#os.chdir(path2)  # Change the directory to the O drive


### Unhide all sheets and define the source extracter (this should run only once) 

In [11]:
%%time 
#for file in glob.glob(r"*.xls*"):  
for file in glob.glob("ANH03 PR24 Data Tables ANH resubmission 10.11.2023_Amended.xlsx"): 
    hidden_sheet_names = ['fOut_OUT', 'fOut_OUT8', 'fOut_RR', 'fOut_CW', 'fOut_CW_MAN', 'fOut_CWW', 'fOut_CWW_MAN', 'fOut_RES', 'fOut_BIO', 'fOut_RET', 'fOut_RET_MAN', 'fOut_DS', 'fOut_DS_MAN', 'fOut_LS', 'fOut_LS_MAN', 'fOut_SUP', 'fOut_SUP_MAN', 'fOut_SUM', 'fOut_PD', 'fOut_PD_MAN', 'fOut_APR']
    wb = load_workbook(file)
    
    # Iterate through each hidden sheet and unhide it
    for sheet_name in hidden_sheet_names:
        ws = wb[sheet_name]
        ws.sheet_state = "visible"
    # Save the changes
    wb.save(file)
    wb.close()    
   
    

CPU times: total: 10min 42s
Wall time: 14min 48s


In [3]:
def find_cell_source(foutput):
    sources = [] #Create a list to store our data in
    for row in foutput.iter_rows(min_row=4, min_col=2, max_col=2):
        for cell in row:
            formula = str(cell.value)
            if formula.startswith("=") and formula.count("'") >= 2:
                #Check if the formula starts with a single quote and contains at least two single quotes
                start_index = formula.find("'") + 1  # Find the first single quote
                end_index = formula.find("'", start_index)  # Find the second single quote
                referenced_sheet = formula[start_index:end_index] 
            elif formula.startswith("=") and formula.count("!")>0 and formula.count("(")>0:
                start_index = formula.find("(") + 1 
                end_index = formula.find("!", start_index)  
                referenced_sheet = formula[start_index:end_index].replace("'", "")
            else:
                # If the formula doesn't follow the expected pattern, set referenced_sheet to None
                referenced_sheet = formula.replace("=", "")
            sources.append(referenced_sheet)
    source_series = sources
    return source_series

### Main Function that will create the Aggragated All.xlsx file

In [4]:
%%time 
# Create an empty list to store data
data = []
df = pd.DataFrame()

#  find sheet names containing "fOut" in a case-insensitive manner 
sheet_name_pattern = re.compile(r'fOut', re.I)

source = []

#for file in glob.glob(r"*.xls*"): 
for file in glob.glob("ANH03 PR24 Data Tables ANH resubmission 10.11.2023_Amended.xlsx"): 
    workbook = openpyxl.load_workbook(file, data_only=True)
    formula_workbook = openpyxl.load_workbook(file, data_only=False)
    acronym = file[:3]
    for sheet_name in workbook.sheetnames:
        if sheet_name_pattern.search(sheet_name):
            worksheet = workbook[sheet_name]
            source = source + find_cell_source(formula_workbook[sheet_name])
            # Create a list to store headers
            headers = []
            for row_index, row in enumerate(worksheet.iter_rows(values_only=True), start=1):
                if row_index == 2:  # Process the second row as headers
                    headers = [cell for cell in row]
                elif row_index != 1 and row_index != 3:  # Skip the first and third row
                    # Create a dictionary to store row data
                    row_data = {'Acronym': acronym, 'Sheet_name': sheet_name}
                    for header, cell in zip(headers, row):
                        row_data[header] = cell
                    # Explicitly set 'Acronym' to an file[:3] if it's None
                    if row_data['Acronym'] is None:
                        row_data['Acronym'] = file[:3]
                    data.append(row_data)
        
df = pd.DataFrame(data)
bpt_series = pd.Series(source)

CPU times: total: 60 s
Wall time: 1min 2s


In [5]:
df["BPT"] = bpt_series

In [6]:
# Reorder and select columns
selected_columns = ["Acronym", "Reference", "Sheet_name","BPT", "Item description", "Unit", "Model", 'Constant', # Reorder the years columns
         "2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24", "2024-25", "2025-26", "2026-27", "2027-28", "2028-29", "2029-30", "2030-31", "2031-32", "2032-33", "2033-34", "2034-35",
         "2020-25", "2025-30", "2030-35", "2035-40", "2040-45", "2045-50",
         '2039-40', '2044-45', '2025-55', '2049-50']

df = df[selected_columns]

# Reset the index
df.reset_index(drop=True, inplace=True)

### Aggragated file with All F_outputs

In [7]:
# Export the results to excel
df.to_excel(r"C:\Users\Jennifer.Tossell\OneDrive - OFWAT\Documents\Coding\All.xlsx", sheet_name="F_Outputs", index=False)

In [ ]:
# Export the results to excel
#df.to_excel(r"O:\OFWSHARE\PR24\231113 - Business Plan Submissions\3_Files error fixing WORKING\Core\Aggregated outputs\Jen's Aggregated outputs\All_BPT_Extracts.xlsx", sheet_name="F_Outputs", index=False)

In [6]:
#acronym - Here write the acronym you want to replace
#new_acronym - Here write what you want to replace it with

def acronym_fix(acronym, new_acronym, cleaned_df):
    
    #First replace the Acronym column with the correction
    temp1 = cleaned_df['Acronym'].replace(acronym, new_acronym)
    cleaned_df['Acronym'] = temp1
    
    #Now replace boncodes containing this acronym with the correction
    temp2 = cleaned_df['Reference'].str.replace(acronym, new_acronym, regex=True)
    cleaned_df['Reference'] = temp2
    
    return cleaned_df

In [7]:
def cleaning_data(df):
    # List of column names where you want to apply data cleaning
    columns_to_replace = ["2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24", "2024-25", "2025-26", "2026-27", "2027-28", "2028-29", "2029-30", "2030-31", "2031-32", "2032-33", "2033-34", "2034-35",
         "2020-25", "2025-30", "2030-35", "2035-40", "2040-45", "2045-50",
         '2039-40', '2044-45', '2025-55', '2049-50']
    problem_cells = ['NR', 'not found', 'No historic data', 'NA', 'N/A', 'n/a', ']', 'N/a']

    # Data Cleaning
    for column in columns_to_replace:
        df[column] = df[column].replace('#DIV/0!', '', regex=True)
        df[column] = df[column].replace({True: 1, False: 0})
        
    # Replace values in problem_cells with an empty string
    df.replace(problem_cells, '', inplace=True)
    
    return df

### Data Cleaning on Glob All.xlsx (replace #DIV/0!, True: 1, False: 0, 'NR', 'not found', 'No historic data', 'NA', 'N/A', 'n/a', ']', 'N/a' to empty cells

In [11]:
%%time 
#Data Cleaning###########################
cleaned_df = acronym_fix('UUW','NWT',df)
cleaned_df=cleaning_data(cleaned_df)

# Export the results to excel
cleaned_df.to_excel(r"O:\OFWSHARE\PR24\231113 - Business Plan Submissions\2_Files after Panacea\Core\Aggregated outputs\\" + "All_data_clean.xlsx", sheet_name="F_Outputs", index=False)

CPU times: total: 2min 19s
Wall time: 2min 45s
